In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
Image_size=256
Batch_size=32
CHANNELS=3

In [5]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/env',
    shuffle=True,
    image_size=(Image_size,Image_size),
    batch_size=Batch_size
)

Found 4085 files belonging to 3 classes.


In [6]:
class_names = dataset.class_names
class_names

['Average_polluted', 'Highly_polltuted', 'Mild_polluted']

In [7]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[0 2 0 1 0 1 1 0 0 1 1 2 0 2 2 0 1 2 1 1 1 1 0 0 2 1 2 0 1 2 1 0]


In [8]:
train_size = 0.8
len(dataset)*train_size


102.4

In [9]:
train_ds = dataset.take(102)
len(train_ds)

102

In [10]:
test_ds = dataset.skip(102)
len(test_ds)

26

In [11]:
val_size=0.1
len(dataset)*val_size

12.8

In [12]:
val_ds = test_ds.take(12)
len(val_ds)

12

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [15]:
len(train_ds)

102

In [16]:
len(val_ds)

12

In [17]:
len(test_ds)

14

In [18]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [19]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [20]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(Image_size, Image_size),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [21]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [22]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [23]:
input_shape = (Batch_size,Image_size, Image_size, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [25]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [26]:
history = model.fit(
    train_ds,
    batch_size=Batch_size,
    validation_data=val_ds,
    verbose=1,
    epochs=200,
)

Epoch 1/200
102/102 [==============================] - 525s 350ms/step - loss: 0.9829 - accuracy: 0.4949 - val_loss: 1.0583 - val_accuracy: 0.4557
Epoch 2/200
102/102 [==============================] - 23s 229ms/step - loss: 0.9267 - accuracy: 0.5321 - val_loss: 1.0956 - val_accuracy: 0.4688
Epoch 3/200
102/102 [==============================] - 24s 240ms/step - loss: 0.9018 - accuracy: 0.5463 - val_loss: 0.9695 - val_accuracy: 0.5052
Epoch 4/200
102/102 [==============================] - 23s 225ms/step - loss: 0.8833 - accuracy: 0.5687 - val_loss: 1.2147 - val_accuracy: 0.4818
Epoch 5/200
102/102 [==============================] - 24s 240ms/step - loss: 0.8696 - accuracy: 0.5724 - val_loss: 0.9118 - val_accuracy: 0.5443
Epoch 6/200
102/102 [==============================] - 23s 224ms/step - loss: 0.8575 - accuracy: 0.5832 - val_loss: 0.9069 - val_accuracy: 0.5521
Epoch 7/200
102/102 [==============================] - 24s 238ms/step - loss: 0.8409 - accuracy: 0.5838 - val_loss: 1.0302 

In [1]:
scores = model.evaluate(test_ds)


NameError: ignored

In [ ]:
scores

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'],color='orange',label='training loss')
plt.plot(history.history['val_loss'],label='validation los')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(18, 10))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot( val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')